In [ ]:
%run config.py

In [ ]:
import random
def rev(qbits,qbit):
	return qbits-qbit-1

def randomgate(qbits):
	op=random.choice(['x','y','z','h','cx','cz','rx','ry','rz'])
	if op=='cx' or op=='cz':
		# Random generation
		q1=random.choice(range(qbits))
		q2=random.choice(range(qbits))
		while q1==q2:
			q2=random.choice(range(qbits))

		# Operation names for different simulators
		if op=='cx':
			questOp='controlledNot'
			pennyOp='CNOT'
		if op=='cz':
			questOp='controlledPhaseFlip'
			pennyOp='CZ'

		# Operation strings for different simulators
		bmsimString = op+' q'+str(q1)+',q'+str(q2)
		quiskitString = 'qc.'+op+'('+str(q1)+','+str(q2)+')'
		questString = questOp+'(qubits, '+str(rev(qbits,q1))+', '+str(rev(qbits,q2))+');'
		pennylaString = 'qml.'+pennyOp+'(wires=["q'+str(q1)+'","q'+str(q2)+'"])'
		return quiskitString, bmsimString, questString, pennylaString
	elif op=='x' or op=='y' or op=='z' or op=='h':
		# Random generation
		q1=random.choice(range(qbits))

		# Operation names for different simulators
		if op=='h':
			questOp='hadamard'
			pennyOp='Hadamard'
		if op=='x':
			questOp='pauliX'
			pennyOp='PauliX'
		if op=='y':
			questOp='pauliY'
			pennyOp='PauliY'
		if op=='z':
			questOp='pauliZ'
			pennyOp='PauliZ'

		# Operation strings for different simulators
		bmsimString = op+' q'+str(q1)
		quiskitString = 'qc.'+op+'('+str(q1)+')'
		questString = questOp+'(qubits, '+str(rev(qbits,q1))+');'
		pennylaString = 'qml.'+pennyOp+'(wires="q'+str(q1)+'")'
		return quiskitString, bmsimString, questString, pennylaString
	elif op=='rx' or op=='ry' or op=='rz':
		# Random generation
		q1=random.choice(range(qbits))
		angle=random.random()*2*3.14159265359

		# Operation names for different simulators
		if op=='rx':
			questOp='rotateX'
			pennyOp='RX'
		if op=='ry':
			questOp='rotateY'
			pennyOp='RY'
		if op=='rz':
			questOp='rotateZ'
			pennyOp='RZ'

		# Operation strings for different simulators
		bmsimString = op+' q'+str(q1)+','+str(angle)
		quiskitString = 'qc.'+op+'('+str(angle)+','+str(q1)+')'
		questString = questOp+'(qubits, '+str(rev(qbits,q1))+', '+str(angle)+');'
		pennylaString = 'qml.'+pennyOp+'('+str(angle)+', wires="q'+str(q1)+'")'
		return quiskitString, bmsimString, questString, pennylaString
	 

In [ ]:
def randomcircuit(qbits, gates):
	qiscirc=""
	bmcirc=""
	qcirc=""
	pcirc='\tqml.QubitStateVector(state, wires=qList)\n'
	for i in range(gates):
		qiscirct, bmcirct, qcirct, pcirct = randomgate(qbits)
		qiscirc+=qiscirct+'\n'
		bmcirc+='\t'+bmcirct+'\n'
		qcirc+=qcirct+'\n'
		pcirc+='\t'+pcirct+'\n'
	pcirc+='\treturn qml.state()\n'
	return qiscirc, bmcirc, qcirc, pcirc

In [ ]:
def randomsim(qbits,gates):
	qbitslist=""
	for i in range(qbits):
		if i==0:
			qbitslist+="q"+str(i)
		else:
			qbitslist+=",q"+str(i)
	qiscirc, bmcirc, qcirc , pcirc = randomcircuit(qbits, gates)
	with open('qiskit/circuit.py.tmpl','r') as f:
		qiskitscript=f.read()

	with open('bmqsim/program.bmq.tmpl','r') as f:
		bmprogram=f.read()

	with open('quest/circuit.c.tmpl','r') as f:
		qprogram=f.read()
	
	with open('pennylane/circuit.py.tmpl','r') as f:
		pennylanescript=f.read()

	qiskitscript=qiskitscript.replace('%%CIRCUIT%%',qiscirc).replace('%%QBITS%%',str(qbits)).replace('%%QBITSLIST%%',qbitslist)
	bmprogram=bmprogram.replace('%%CIRCUIT%%',bmcirc).replace('%%QBITS%%',str(qbits)).replace('%%QBITSLIST%%',qbitslist)
	qprogram=qprogram.replace('%%CIRCUIT%%',qcirc).replace('%%QBITS%%',str(qbits)).replace('%%QBITSLIST%%',qbitslist)
	pennylanescript=pennylanescript.replace('%%CIRCUIT%%',pcirc).replace('%%QBITS%%',str(qbits)).replace('%%QBITSLIST%%',qbitslist)

	with open('qiskit/circuit.py','w') as f:
		f.write(qiskitscript)

	with open('bmqsim/program.bmq','w') as f:
		f.write(bmprogram)

	with open('quest/circuit.c','w') as f:
		f.write(qprogram)

	with open('pennylane/circuit.py','w') as f:
		f.write(pennylanescript)

	%run analyze.ipynb

In [ ]:
randomsim(qbits,50)
if passed:
	print('Test passed')
else:
	print('Test failed')